In [3]:
import pandas as pd
import re 

In [116]:
df = pd.read_csv("ClientParticulars (Candidate_s Copy).csv")
df

,FirstName,LastName,Gender,Age,MarriageStatus,PhoneNumber,NetYearlyIncome,HighValueIndividual
0,Maisy,O'Doherty,NaN,NaN,NaN,NaN,NaN,NaN
1,Maisy,O'Doherty,Female,26.0,Single,488 238-8338maisyodoherty@gmail.com,73000.0,No
2,Ellis,Ross,Female,66.0,Married,(830) 694-4738ellisross@gmail.com,176000.0,No
3,Dana,Oconnor,Female,52.0,Married,(296)855-2148danaoconnor@gmail.com,266000.0,No
4,Gareth,Ferrell,Male,46.0,Single,882400-9859garethferrell@gmail.com,729000.0,Yes
...,...,...,...,...,...,...,...,...
105,Kaya,Carrillo,Male,31.0,Single,(314)431-7751kayacarrillo@gmail.com,252000.0,No
106,Cormac,Moyer,Male,40.0,Married,(609) 350-3771cormacmoyer@gmail.com,78000.0,No
107,Cade,Watts,Male,NaN,Married,(827) 222-0618cadewatts@gmail.com,645000.0,NaN
108,Evelyn,Brandt,Female,58.0,Married,(930) 546-6988evelynbrandt@gmail.com,470000.0,Yes


In [117]:
# drop rows without first and last names (unable to fill)
df = df.dropna(subset=["FirstName", "LastName"])

# drop rows without any data, just names
df = df.dropna(subset=df.columns[2:], how="all")

# df = df.drop_duplicates(subset=["FirstName", "LastName"])

df.head(10)

,FirstName,LastName,Gender,Age,MarriageStatus,PhoneNumber,NetYearlyIncome,HighValueIndividual
1,Maisy,O'Doherty,Female,26.0,Single,488 238-8338maisyodoherty@gmail.com,73000.0,No
2,Ellis,Ross,Female,66.0,Married,(830) 694-4738ellisross@gmail.com,176000.0,No
3,Dana,Oconnor,Female,52.0,Married,(296)855-2148danaoconnor@gmail.com,266000.0,No
4,Gareth,Ferrell,Male,46.0,Single,882400-9859garethferrell@gmail.com,729000.0,Yes
5,Jaime,Boone,Male,NaN,Married,(569) 6117858jaimeboone@gmail.com,460000.0,Yes
6,Jemma,Sloan,Female,41.0,Married,(730)864-9016jemmasloan@gmail.com,480000.0,Yes
7,Blake,Monroe,Male,73.0,NaN,(903) 899-6641blakemonroe@gmail.com,427000.0,NaN
8,Lilly-Rose,Humphrey,Female,29.0,Single,(897) 854-9070lilly-rosehumphrey@gmail.com,368000.0,Yes
9,Ruairi,Schmidt,Male,30.0,Single,(766) 325-9938ruairischmidt@gmail.com,936000.0,Yes
10,Cody,Drake,Male,56.0,Married,(740) 812-6242codydrake@gmail.com,283000.0,No


In [118]:
df.loc[df["HighValueIndividual"]=="Yes", "NetYearlyIncome"].min()

311000.0

In [119]:
df.loc[df["HighValueIndividual"]=="No", "NetYearlyIncome"].max()

283000.0

In [120]:
# Assume HighValueIndividual cutoff = 300000
# Fill HighValueIndividual affected by NetYearlyIncome
high_value_indiv_thresh = 300000

df.loc[(df["HighValueIndividual"].isna()) & (df["NetYearlyIncome"]>=high_value_indiv_thresh), "HighValueIndividual"] = "Yes"
df.loc[(df["HighValueIndividual"].isna()) & (df["NetYearlyIncome"]<high_value_indiv_thresh), "HighValueIndividual"] = "No"

In [121]:
df.loc[df["MarriageStatus"]=="Married", "Age"].mean()

54.75925925925926

In [122]:
df.loc[df["MarriageStatus"]=="Single", "Age"].mean()

33.733333333333334

In [123]:
# Fill age
df.loc[(df["Age"].isna()) & (df["MarriageStatus"]=="Married"), "Age"] = int(df.loc[df["MarriageStatus"]=="Married", "Age"].mean())
df.loc[(df["Age"].isna()) & (df["MarriageStatus"]=="Single"), "Age"] = int(df.loc[df["MarriageStatus"]=="Single", "Age"].mean())

In [124]:
df.isna().sum()

FirstName               0
LastName                0
Gender                  0
Age                     2
MarriageStatus         11
PhoneNumber             0
NetYearlyIncome         0
HighValueIndividual     0
dtype: int64

In [125]:
df = df.dropna(subset=["Age"])
df.isna().sum()

FirstName              0
LastName               0
Gender                 0
Age                    0
MarriageStatus         9
PhoneNumber            0
NetYearlyIncome        0
HighValueIndividual    0
dtype: int64

In [126]:
df.loc[df["MarriageStatus"].isna()]

,FirstName,LastName,Gender,Age,MarriageStatus,PhoneNumber,NetYearlyIncome,HighValueIndividual
7,Blake,Monroe,Male,73.0,NaN,(903) 899-6641blakemonroe@gmail.com,427000.0,Yes
27,Flynn,Bloggs,Female,68.0,NaN,(722) 551-5493flynnbloggs@gmail.com,74000.0,No
39,Teddy,Hebert,Male,49.0,NaN,(816) 750-8753teddyhebert@gmail.com,205000.0,No
50,Kade,Townsend,Female,77.0,NaN,(229) 283-5145kadetownsend@gmail.com,784000.0,Yes
60,Darcey,Riley,Female,69.0,NaN,(359) 397-1549darceyriley@gmail.com,513000.0,Yes
63,Zackary,Dennis,Male,50.0,NaN,634 649-5998zackarydennis@gmail.com,494000.0,Yes
73,Carmen,Irwin,Female,23.0,NaN,(882)949-8211carmenirwin@gmail.com,808000.0,Yes
75,Cynthia,Herrera,Female,44.0,NaN,(569) 740-3579cynthiaherrera@gmail.com,85000.0,No
89,Giovanni,Gutierrez,Male,52.0,NaN,(734)5136684giovannigutierrez@gmail.com,804000.0,Yes


In [127]:
df.loc[df["MarriageStatus"]=="Single", "Age"].mean()

33.666666666666664

In [128]:
df.loc[df["MarriageStatus"]=="Married", "Age"].mean()

54.68333333333333

In [134]:
# Fill MarriageStatus
mean_single_age = df.loc[df["MarriageStatus"]=="Single", "Age"].mean()
mean_married_age = df.loc[df["MarriageStatus"]=="Married", "Age"].mean()

def fillMarriageStatus(age):
    if abs(age-mean_single_age) < abs(age-mean_married_age):
        return "Single"
    else:
        return "Married"

df["MarriageStatus"] = df["Age"].apply(lambda x: fillMarriageStatus(x))
df

,FirstName,LastName,Gender,Age,MarriageStatus,PhoneNumber,NetYearlyIncome,HighValueIndividual
1,Maisy,O'Doherty,Female,26.0,Single,488 238-8338maisyodoherty@gmail.com,73000.0,No
2,Ellis,Ross,Female,66.0,Married,(830) 694-4738ellisross@gmail.com,176000.0,No
3,Dana,Oconnor,Female,52.0,Married,(296)855-2148danaoconnor@gmail.com,266000.0,No
4,Gareth,Ferrell,Male,46.0,Married,882400-9859garethferrell@gmail.com,729000.0,Yes
5,Jaime,Boone,Male,54.0,Married,(569) 6117858jaimeboone@gmail.com,460000.0,Yes
...,...,...,...,...,...,...,...,...
105,Kaya,Carrillo,Male,31.0,Single,(314)431-7751kayacarrillo@gmail.com,252000.0,No
106,Cormac,Moyer,Male,40.0,Single,(609) 350-3771cormacmoyer@gmail.com,78000.0,No
107,Cade,Watts,Male,54.0,Married,(827) 222-0618cadewatts@gmail.com,645000.0,Yes
108,Evelyn,Brandt,Female,58.0,Married,(930) 546-6988evelynbrandt@gmail.com,470000.0,Yes


In [137]:
df = df.reset_index(drop=True)
df.isna().sum()

FirstName                0
LastName                 0
Gender                   0
Age                      0
MarriageStatus           0
PhoneNumber              0
NetYearlyIncome          0
HighValueIndividual      0
IsHighValueIndividual    0
dtype: int64

In [159]:
def getEmail(phone_num):
    return re.split("\d", phone_num)

df["EmailAddress"] = df["PhoneNumber"].apply(lambda x: getEmail(x)[-1].strip())

In [173]:
def getPhone(phone_num):
    return re.split("(?!.*\d)", phone_num)

df["PhoneNumber"] = df["PhoneNumber"].apply(lambda x: getPhone(x)[0])

In [174]:
df

,FirstName,LastName,Gender,Age,MarriageStatus,PhoneNumber,NetYearlyIncome,HighValueIndividual,IsHighValueIndividual,EmailAddress
0,Maisy,O'Doherty,Female,26.0,Single,488 238-8338,73000.0,No,0,maisyodoherty@gmail.com
1,Ellis,Ross,Female,66.0,Married,(830) 694-4738,176000.0,No,0,ellisross@gmail.com
2,Dana,Oconnor,Female,52.0,Married,(296)855-2148,266000.0,No,0,danaoconnor@gmail.com
3,Gareth,Ferrell,Male,46.0,Married,882400-9859,729000.0,Yes,1,garethferrell@gmail.com
4,Jaime,Boone,Male,54.0,Married,(569) 6117858,460000.0,Yes,1,jaimeboone@gmail.com
...,...,...,...,...,...,...,...,...,...,...
97,Kaya,Carrillo,Male,31.0,Single,(314)431-7751,252000.0,No,0,kayacarrillo@gmail.com
98,Cormac,Moyer,Male,40.0,Single,(609) 350-3771,78000.0,No,0,cormacmoyer@gmail.com
99,Cade,Watts,Male,54.0,Married,(827) 222-0618,645000.0,Yes,1,cadewatts@gmail.com
100,Evelyn,Brandt,Female,58.0,Married,(930) 546-6988,470000.0,Yes,1,evelynbrandt@gmail.com
